<div style="font-size:18pt; padding-top:20px; text-align:center"><b>Лабораторная работа 2.</b> Разработка и запуск MapReduce приложений</div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_bmstu@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Разработка приложений MapReduce с ипользованием IDE Eclipse</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#1a">Исходный код программы</a></li>
                <li><a href="#1b">Запуск проекта</a></li>
            </ol>
        </li>
        <li><a href="#2">Запуск приложения MapReduce</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#2a">Запуск на локальной версии Cloudera</a></li>
                <li><a href="#2b">Запуск на кластере EMR AWS</a></li>
                <li><a href="#2c">Запуск на кластере HDInsight Azure</a></li>
            </ol>
        </li>
        <li><a href="#5">Источники</a>
        </li>
    </ol>
</div>

<div>
    <p><b>Цель лабораторной работы</b> - освоить:</p>
    <ol>
        <li>создание проектов MapReduce с использованием среды разработки Eclipse</li>
        <li>запуск приложений MapReduce на Hadoop кластере</li>
        <li>подключение к средствам мониторинга ресурсов Hadoop кластер</li>
    </ol>
    <p><b>Необходимые средства и схема подключения</b></p>
    <img src="img/structure_tools.png" alt="Tools for Lab 2" width="60%" style="max-width:500px;">
</div>

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Разработка приложений MapReduce с ипользованием IDE Eclipse</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name = "1a"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Исходный код программы
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#1b">Далее</a>
            </div>
        </div>
    </div>
</div>

http://ota.ox.ac.uk/text/5721.txt - датасет

https://hadoop.apache.org/docs/r2.7.0/hadoop-mapreduce-client/hadoop-mapreduce-client-core/MapReduceTutorial.html

In [ ]:
import java.io.IOException;
import java.util.StringTokenizer;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class WordCount {

  public static class TokenizerMapper
       extends Mapper<Object, Text, Text, IntWritable>{

    private final static IntWritable one = new IntWritable(1);
    private Text word = new Text();

    public void map(Object key, Text value, Context context
                    ) throws IOException, InterruptedException {
      StringTokenizer itr = new StringTokenizer(value.toString());
      while (itr.hasMoreTokens()) {
        word.set(itr.nextToken());
        context.write(word, one);
      }
    }
  }

  public static class IntSumReducer
       extends Reducer<Text,IntWritable,Text,IntWritable> {
    private IntWritable result = new IntWritable();

    public void reduce(Text key, Iterable<IntWritable> values,
                       Context context
                       ) throws IOException, InterruptedException {
      int sum = 0;
      for (IntWritable val : values) {
        sum += val.get();
      }
      result.set(sum);
      context.write(key, result);
    }
  }

  public static void main(String[] args) throws Exception {
    Configuration conf = new Configuration();
    Job job = Job.getInstance(conf, "word count");
    job.setJarByClass(WordCount.class);
    job.setMapperClass(TokenizerMapper.class);
    job.setCombinerClass(IntSumReducer.class);
    job.setReducerClass(IntSumReducer.class);
    job.setOutputKeyClass(Text.class);
    job.setOutputValueClass(IntWritable.class);
    FileInputFormat.addInputPath(job, new Path(args[0]));
    FileOutputFormat.setOutputPath(job, new Path(args[1]));
    System.exit(job.waitForCompletion(true) ? 0 : 1);
  }
}

Исходный код программы на Scala (см. также https://github.com/twitter/scalding )

In [ ]:
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.fs.Path
import org.apache.hadoop.io.IntWritable
import org.apache.hadoop.io.Text
import org.apache.hadoop.mapreduce.Job
import org.apache.hadoop.mapreduce.Mapper
import org.apache.hadoop.mapreduce.Reducer
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat
import org.apache.hadoop.util.GenericOptionsParser
import scala.collection.JavaConversions._

// This class performs the map operation, translating raw input into the key-value
// pairs we will feed into our reduce operation.
class TokenizerMapper extends Mapper[Object, Text, Text, IntWritable] {
  val one = new IntWritable(1)
  val word = new Text

  override
  def map(key: Object, value: Text, context: Mapper[Object, Text, Text, IntWritable]#Context) = {
    for (t <- value.toString.split("\\s")) {
      word.set(t)
      context.write(word, one)
    }
  }
}

// This class performs the reduce operation, iterating over the key-value pairs
// produced by our map operation to produce a result. In this case we just
// calculate a simple total for each word seen.
class IntSumReducer extends Reducer[Text, IntWritable, Text, IntWritable] {
  override
  def reduce(key: Text, values: java.lang.Iterable[IntWritable], context: Reducer[Text, IntWritable, Text, IntWritable]#Context) = {
    val sum = values.foldLeft(0) { (t, i) => t + i.get }
    context.write(key, new IntWritable(sum))
  }
}

// This class configures and runs the job with the map and reduce classes we've
// specified above.
object WordCount extends App {

  val conf = new Configuration()
  val otherArgs = new GenericOptionsParser(conf, args).getRemainingArgs
  if (otherArgs.length != 2) {
    println("Usage: wordcount <in> <out>")
    sys.exit(2)
  }
  val job = Job.getInstance(conf, "word count")
  job.setJarByClass(classOf[TokenizerMapper])
  job.setMapperClass(classOf[TokenizerMapper])
  job.setCombinerClass(classOf[IntSumReducer])
  job.setReducerClass(classOf[IntSumReducer])
  job.setOutputKeyClass(classOf[Text])
  job.setOutputValueClass(classOf[IntWritable])
  FileInputFormat.addInputPath(job, new Path(args(0)))
  FileOutputFormat.setOutputPath(job, new Path(args(1)))
  sys.exit(if (job.waitForCompletion(true)) 0 else 1)
}

Скрипт для сборки Scala проекта:

In [ ]:
name := "map_reduce"
 
version := "1.0"
 
scalaVersion := "2.11.8"

libraryDependencies += "org.apache.hadoop" % "hadoop-client" % "2.7.3"
libraryDependencies += "org.scalatest" %% "scalatest" % "3.0.0" % "test"

assemblyMergeStrategy in assembly := {
  case PathList("META-INF", "MANIFEST.MF") => MergeStrategy.discard
  case _ => MergeStrategy.first
}


<a name = "1b"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Запуск проекта
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#1b">Далее</a>
            </div>
        </div>
    </div>
</div>

<p>Самостоятельное добавлние пакетов Hadoop в проект</p>

Java -> JAR <br>
https://www.cs.utexas.edu/~scottm/cs307/handouts/Eclipse%20Help/jarInEclipse.htm

Сборка зависимостей с использованием Maven

http://tutorials.pcdhan.com/hadoop/developing-your-first-mapreduce-app-in-java-using-eclipse-and-maven/

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Запуск приложения MapReduce</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name = "2a"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Запуск на локальной версии Cloudera
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#2b">Далее</a>
            </div>
        </div>
    </div>
</div>

<div>
    <p><b>Схема подключения</b></p>
    <img src="img/connection_cloudera.png" alt="Tools for Lab 2" width="30%" style="max-width:300px;min-width:200px;">
</div>

<p><b>Подготовка исходных файлов с данными и выполняемой программой MapReduce</b></p>

<p>Копирование файла с исходными данными на узел кластера</p>

<p style="border: 1px solid lightgrey; padding: 5px 5px 5px 5px; font-family: Courier New">
<span style="color: green; font-weight: bold">scp</span>
/home/sergo/downloads/Datasets/ROMEO_and_IVLIET.txt
cloudera@192.168.42.29:/home/cloudera/wiil.csv</p>

In [ ]:
scp /home/sergo/downloads/Datasets/ROMEO_and_IVLIET.txt cloudera@192.168.42.29:/home/cloudera/wiil.csv

<p>Копирование выполняемого файла jar на узел кластера</p>

In [ ]:
scp /home/sergo/downloads/hadoop_mr-0.0.1-SNAPSHOT.jar cloudera@192.168.42.29:/home/cloudera/hadoop_mr-0.0.1-SNAPSHOT.jar

<p>Подключение к кластеру</p>

In [ ]:
sudo ssh 192.168.42.29

<p>Копирование файла с исходными данными из локальной папки в HDFS </p>

In [ ]:
hdfs dfs -copyFromLocal /home/cloudera/wiil.csv /user/root

<p><b>Запуск приложения MapReduce</b></p>

<p>Команда запуска</p>

In [ ]:
hadoop jar /home/cloudera/hadoop_mr-0.0.1-SNAPSHOT.jar bigdata.labs.WordCount /user/root/wiil.csv /user/root/output

<p><b>Проверка выполнения</b></p>

<p><i>Проверка выходных файлов в HDFS</i></p>

<p>Отображение списка папок и файлов HDFS</p>

In [ ]:
hdfs dfs -ls /user/root

<p>Рекурсивное отображение списка папок и файлов HDFS</p>

In [ ]:
hdfs dfs -ls -R /user/root

<p>Проверка наличия выходных файлов в папке HDFS "/user/root/output"</p>

In [ ]:
hdfs dfs -ls /user/root/output

In [ ]:
hadoop fs -ls /user/root/output

<p><i>Подключение к панели ResourсeManager</i></p>

<a name = "2b"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Запуск на кластере EMR AWS  
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2a">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#2c">Далее</a>
            </div>
        </div>
    </div>
</div>

<div>
    <p><b>Схема подключения</b></p>
    <img src="img/connection_aws.png" alt="Tools for Lab 2" width="50%" style="max-width:500px;min-width:300px;">
</div>
</div>

<p><b>Развертывание кластера EMR</b></p>

<a href="http://docs.aws.amazon.com/ElasticMapReduce/latest/DeveloperGuide/gsg-launch-cluster.html">Launch an Amazon EMR Cluster</a>

<p>Подключение к кластеру</p>

In [ ]:
%%bash
CLUSTER_DNS=YOUR_DNS
KEY_PATH=YOUR_KEY.pem

ssh -i $KEY_PATH hadoop@$CLUSTER_DNS #ssh connection
hdfs version #hdfs version

<p>Проверка соединения</p>

In [ ]:
hdfs version

<p>Копирование папки с исходными файлами на машину кластера ERM</p>

In [ ]:
scp -i YOUR_KEY.pem -rp /home/sergo/downloads/WordCount-Files hadoop@YOUR_DNS:/tmp/mr-job

<p>Копирование файла с исходными данными из локальной папки в HDFS </p>

In [ ]:
hdfs dfs -copyFromLocal /tmp/mr-job/ROMEO_and_IVLIET.txt /user/root

<p>Проверка записи файла</p>

In [ ]:
hdfs dfs -ls /user/root

In [ ]:
hdfs fsck /user/root/ROMEO_and_IVLIET.txt -files -blocks -locations

<p>Запуск приложения MapReduce</p>

In [ ]:
hadoop jar /tmp/mr-job/hadoop_mr-0.0.1-SNAPSHOT.jar bigdata.labs.WordCount /user/root/ROMEO_and_IVLIET.txt /user/root/output

<p>Проверка выполнения</p>

In [ ]:
hdfs dfs -ls /user/root/output

In [ ]:
hdfs dfs -tail /user/root/output/part-r-00000

<p><b>Развертывание кластера EMR и запуск приложения посредством AWS CLI</b></p>

<p>Конфигурирование доступа к кластеру через AWS CLI</p>

In [ ]:
%%bash
KEY_ID=YOUR_KEY
ACCESS_KEY=YOUR_ACCES
REGION=YOUR_REGION

aws configure set aws_access_key_id $KEY_ID
aws configure set aws_secret_access_key $ACCESS_KEY
aws configure set region $REGION

<p>Отображение бакетов S3</p>

In [ ]:
%%bash
aws s3 ls --recursive --human-readable

<p>Создание нового бакета</p>

In [ ]:
%%bash
aws s3 mb s3://aws-mr-jobs-labs/

<p>Копирование файлов с исходными данными из локальной папки в S3</p>

In [ ]:
%%bash
aws s3 cp /home/sergo/downloads/WordCount-Files s3://aws-mr-jobs-labs/lab2 --recursive

<p>Развертывание кластера и запуск приложения MapReduce</p>
<p></p>

http://docs.aws.amazon.com/ElasticMapReduce/latest/DeveloperGuide/emr-launch-custom-jar-cli.html

In [ ]:
%%bash
aws emr create-cluster --name "Hadoop_WordCount" \
--ami-version 3.11.0 \
--log-uri s3://aws-mr-jobs-labs/lab2/logs/ \
--use-default-roles \
--ec2-attributes KeyName=hadoop_keys,SubnetId=subnet-<your-subnet-id> \
--instance-type m4.large \
--instance-count 3 \
--
--steps 'Type=CUSTOM_JAR,Name="WordCount_JAR_Step",ActionOnFailure=CONTINUE,Jar=s3://aws-mr-jobs-labs/lab2/hadoop_mr-0.0.1-SNAPSHOT.jar,MainClass=bigdata.labs.WordCount,Args=["s3://aws-mr-jobs-labs/lab2/ROMEO_and_IVLIET.txt","s3://aws-mr-jobs-labs/lab2/output"]'
#--applications Name=Hue Name=Hive Name=Pig \

<p>Проверка выполнения</p>

In [ ]:
%%bash
aws s3 ls s3://aws-mr-jobs-labs/lab2/output/

In [ ]:
%%bash
aws s3 cp s3://aws-mr-jobs-labs/lab2/output/part-r-00000 /home/sergo/downloads/part-r-00000

In [ ]:
%%bash
tail /home/sergo/downloads/part-r-00000

<p>Подключение к панели ResourceManager</p>

<p>Удаление папки с резульататом работы MapReduce</p>

In [ ]:
%%bash
aws s3 rm s3://aws-mr-jobs-labs/lab2/output --recursive

<a name = "2с"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            с. Запуск на кластере HDInsight Azure  
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2b">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#3">Далее</a>
            </div>
        </div>
    </div>
</div>

<b>Запуск с хоста в кластере</b>

Подключения к машине в кластере:

In [ ]:
%%bash
# узнать строку подключения можно кликнув на ваш кластер на portal.azure.com (см. вверху вкладку "Безопасная оболочка")
ssh username@user-cluster-ssh.azurehdinsight.net

С локального хоста необходимо скопировать jar-файл с кодом и датасет на машину кластера:

In [ ]:
%%bash

scp wordcount.jar username@user-cluster-ssh.azurehdinsight.net:/home/username
scp data.txt username@user-cluster-ssh.azurehdinsight.net:/home/username

Далее сохраним данные, которые мы собираемся обработать, на распределенной файловой системе (jar-файл в свою очередь доступен по локальному пути)

In [ ]:
%%bash

# альтернативный синтаксис hdfs dfs
hadoop fs -mkdir /user/root/WordCount
hadoop fs -copyFromLocal data.txt /user/root/WordCount/

Теперь с машины в кластере мы можем запустить MapReduce job. Обратите внимение, что MS Azure для хранения использует Blob storage - своего рода обертка над HDFS, поэтому URL до директории с результатами работы программы имеет префикс "wasbs://"

In [ ]:
%%bash

# yarn jar <jar-файл> <имя запускаемого класса> <данные для обработки> <директория с результатами работы>
yarn jar wordcount.jar WordCount \
    wasb:///user/root/WordCount/data.txt wasb:///user/root/WordCount/output

Выполнив команду "hadoop fs -ls /user/root/WordCount/output/" вы увидете 2 файла: SUCCESS (говорит о том, что job выполнился успешно) и part-r-00000 - это и есть файл с результатами, выведем его содержимое на экран

In [ ]:
%%bash

hadoop fs -cat /user/root/WordCount/output/part-r-00000